In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import statistics

In [2]:
fileName = "./scripts/sssp_trails_1_results.csv"
new_data = pd.read_csv(fileName)

In [41]:
max(new_data["Real Time"])

43744.0

In [36]:
for index, row in new_data.iterrows():
    print(int(row["CPU Frequencies"]),int(row["GPU Frequencies"]),int(row["Mem Frequencies"]),round(row["Real Time"],3))

345600 114750000 40800000 43744.0
345600 114750000 68000000 25519.0
345600 114750000 102000000 18363.0
345600 114750000 204000000 12554.0
345600 114750000 408000000 10052.0
345600 114750000 665600000 9480.0
345600 114750000 800000000 9508.0
345600 114750000 1062400000 9371.0
345600 114750000 1331200000 9307.0
345600 114750000 1600000000 9351.0
345600 114750000 1866000000 9232.0
345600 216750000 40800000 39845.0
345600 216750000 68000000 21916.0
345600 216750000 102000000 17205.0
345600 216750000 204000000 11467.0
345600 216750000 408000000 8997.0
345600 216750000 665600000 8146.0
345600 216750000 800000000 7930.0
345600 216750000 1062400000 7785.0
345600 216750000 1331200000 7716.0
345600 216750000 1600000000 7621.0
345600 216750000 1866000000 7590.0
345600 318750000 40800000 37046.0
345600 318750000 68000000 22249.0
345600 318750000 102000000 17128.0
345600 318750000 204000000 11618.0
345600 318750000 408000000 8626.0
345600 318750000 665600000 8258.0
345600 318750000 800000000 7596.0

806400 854250000 665600000 4154.0
806400 854250000 800000000 4033.0
806400 854250000 1062400000 3732.0
806400 854250000 1331200000 3553.0
806400 854250000 1600000000 3484.0
806400 854250000 1866000000 3451.0
806400 930750000 40800000 35827.0
806400 930750000 68000000 19167.0
806400 930750000 102000000 13217.0
806400 930750000 204000000 7745.0
806400 930750000 408000000 5292.0
806400 930750000 665600000 4333.0
806400 930750000 800000000 4098.0
806400 930750000 1062400000 3727.0
806400 930750000 1331200000 3567.0
806400 930750000 1600000000 3446.0
806400 930750000 1866000000 3413.0
806400 1032750000 40800000 35910.0
806400 1032750000 68000000 18466.0
806400 1032750000 102000000 12960.0
806400 1032750000 204000000 7771.0
806400 1032750000 408000000 5209.0
806400 1032750000 665600000 4197.0
806400 1032750000 800000000 3985.0
806400 1032750000 1062400000 4014.0
806400 1032750000 1331200000 3599.0
806400 1032750000 1600000000 3443.0
806400 1032750000 1866000000 3404.0
806400 1122000000 40800

1113600 624750000 408000000 4624.0
1113600 624750000 665600000 3720.0
1113600 624750000 800000000 3467.0
1113600 624750000 1062400000 3238.0
1113600 624750000 1331200000 3027.0
1113600 624750000 1600000000 2919.0
1113600 624750000 1866000000 2882.0
1113600 726750000 40800000 35214.0
1113600 726750000 68000000 18784.0
1113600 726750000 102000000 12796.0
1113600 726750000 204000000 7174.0
1113600 726750000 408000000 4540.0
1113600 726750000 665600000 3671.0
1113600 726750000 800000000 3439.0
1113600 726750000 1062400000 3148.0
1113600 726750000 1331200000 2919.0
1113600 726750000 1600000000 2861.0
1113600 726750000 1866000000 2814.0
1113600 854250000 40800000 33937.0
1113600 854250000 68000000 18719.0
1113600 854250000 102000000 12779.0
1113600 854250000 204000000 7114.0
1113600 854250000 408000000 4562.0
1113600 854250000 665600000 3645.0
1113600 854250000 800000000 3434.0
1113600 854250000 1062400000 3133.0
1113600 854250000 1331200000 2858.0
1113600 854250000 1600000000 2783.0
1113600

In [42]:
def getPerformance(fileName):
    new_data = pd.read_csv(fileName)
    base_perf = max(new_data["Real Time"])
    perf_data = {}
    for index,row in new_data.iterrows():
        cpuF = int(row["CPU Frequencies"])
        gpuF = int(row["GPU Frequencies"])
        memF = int(row["Mem Frequencies"])
        time = round(row["Real Time"],3)
        perf = base_perf / time
        perf_data[(int(cpuF),int(memF),int(gpuF))] = perf
    return perf_data
    

getPerformance("./scripts/sssp_trails_1_results.csv")

{(345600, 40800000, 114750000): 1.0,
 (345600, 68000000, 114750000): 1.7141737528900036,
 (345600, 102000000, 114750000): 2.3821815607471546,
 (345600, 204000000, 114750000): 3.4844671021188467,
 (345600, 408000000, 114750000): 4.351770791882212,
 (345600, 665600000, 114750000): 4.614345991561182,
 (345600, 800000000, 114750000): 4.6007572570466975,
 (345600, 1062400000, 114750000): 4.668018354497919,
 (345600, 1331200000, 114750000): 4.700118190609219,
 (345600, 1600000000, 114750000): 4.678002352689552,
 (345600, 1866000000, 114750000): 4.738301559792028,
 (345600, 40800000, 216750000): 1.0978541849667462,
 (345600, 68000000, 216750000): 1.9959846687351706,
 (345600, 102000000, 216750000): 2.5425167102586457,
 (345600, 204000000, 216750000): 3.8147728263713265,
 (345600, 408000000, 216750000): 4.862065132822051,
 (345600, 665600000, 216750000): 5.369997544807267,
 (345600, 800000000, 216750000): 5.5162673392181585,
 (345600, 1062400000, 216750000): 5.619010918432884,
 (345600, 133120

In [49]:
def powerAnalyze(folder):
    dirpath = os.path.join(os.getcwd() + "/"+folder)
    total_pow = {}
    for filename in os.listdir(dirpath):
        ftemp = filename.split("_")
        cpuF,memF,gpuF = ftemp[ftemp.index('C')+1],ftemp[ftemp.index('M')+1],ftemp[ftemp.index('G')+1]
        f = os.path.join(dirpath, filename)
        file = open(f, 'r')
        lines = file.readlines()
        total_pow[(int(cpuF),int(memF),int(gpuF))] = 0
        mydata = []
        for line in lines:
            line = line.replace(" ",",")
            line = line.split(",")
            value = int(line[line.index('VDD_IN')+1].split("/")[0])
            mydata.append(value)
        
        total_pow[(int(cpuF),int(memF),int(gpuF))] = statistics.mean(mydata)
    return total_pow

MeanPow = powerAnalyze("TegraData")     
print("Done")
            
            
        
        
    

Done


In [51]:
#Combined Data
def getCombinedData():
    perf_file = "./scripts/sssp_trails_1_results.csv"
    pow_folder = "TegraData"
    perf = getPerformance(perf_file)
    powD = powerAnalyze("TegraData") # In this function we use average power, we should try some other metric, like the roof power
    combinedData = {}
    for key in perf.keys():
        combinedData[key] = (perf[key],powD[key])
    return combinedData

getCombinedData()
    

{(345600, 40800000, 114750000): (1.0, 2375.7736625514403),
 (345600, 68000000, 114750000): (1.7141737528900036, 2434.276836158192),
 (345600, 102000000, 114750000): (2.3821815607471546, 2472.6296296296296),
 (345600, 204000000, 114750000): (3.4844671021188467, 2446.686868686869),
 (345600, 408000000, 114750000): (4.351770791882212, 1959),
 (345600, 665600000, 114750000): (4.614345991561182, 2054.037037037037),
 (345600, 800000000, 114750000): (4.6007572570466975, 2134),
 (345600, 1062400000, 114750000): (4.668018354497919, 2228.1875),
 (345600, 1331200000, 114750000): (4.700118190609219, 2327.8481012658226),
 (345600, 1600000000, 114750000): (4.678002352689552, 2718.4),
 (345600, 1866000000, 114750000): (4.738301559792028, 3409.3924050632913),
 (345600, 40800000, 216750000): (1.0978541849667462, 1692.4716981132076),
 (345600, 68000000, 216750000): (1.9959846687351706, 1755.3393939393939),
 (345600, 102000000, 216750000): (2.5425167102586457, 1769.5693430656934),
 (345600, 204000000, 21